In [581]:
import base64
import string

# Implementing base64 encoding/decoding from scratch

In [582]:
base64_symbols = string.ascii_uppercase + string.ascii_lowercase + string.digits + '+/'
base64_symbols
len(base64_symbols)

'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/'

64

In [583]:
for i, symbol in enumerate(base64_symbols):
  print(i, f'{i:06b}', symbol)  # 6 bits = (2^6=64 states)

0 000000 A
1 000001 B
2 000010 C
3 000011 D
4 000100 E
5 000101 F
6 000110 G
7 000111 H
8 001000 I
9 001001 J
10 001010 K
11 001011 L
12 001100 M
13 001101 N
14 001110 O
15 001111 P
16 010000 Q
17 010001 R
18 010010 S
19 010011 T
20 010100 U
21 010101 V
22 010110 W
23 010111 X
24 011000 Y
25 011001 Z
26 011010 a
27 011011 b
28 011100 c
29 011101 d
30 011110 e
31 011111 f
32 100000 g
33 100001 h
34 100010 i
35 100011 j
36 100100 k
37 100101 l
38 100110 m
39 100111 n
40 101000 o
41 101001 p
42 101010 q
43 101011 r
44 101100 s
45 101101 t
46 101110 u
47 101111 v
48 110000 w
49 110001 x
50 110010 y
51 110011 z
52 110100 0
53 110101 1
54 110110 2
55 110111 3
56 111000 4
57 111001 5
58 111010 6
59 111011 7
60 111100 8
61 111101 9
62 111110 +
63 111111 /


**Chunks having less than 6 bits are bad**

Encoding (more than one binary sequence mapping to F):  
'101' -> 5 -> 'F'  
'0101' -> 5 -> 'F'  
'00101' -> 5 -> 'F'  
'000101' -> 5 -> 'F'  

Decoding (F is mapped to 6 bit binary sequence):  
'F' -> 5 -> '000101'  

Data is corrupted after decoding:  
'101' -> '000101'  

In [584]:
# Demonstration:
bad_chunk = '101'  # less than 6 bits
bad_chunk

# Encoding (101 -> F):
int(bad_chunk, 2)  # chunk of 2/3/4/5 bits -> num
base64_symbols[5]  # num -> symbol

# Decoding (F -> 000101):
base64_symbols.index('F')  # symbol -> num
f'{5:06b}'  # num -> chunk of 6 bits

'101'

5

'F'

5

'000101'

In [585]:
good_chunk = '001100'  # 6 bits OK
good_chunk

# Encoding (001100 -> M):
int(good_chunk, 2)
base64_symbols[12]

# Decoding (M -> 001100):
base64_symbols.index('M')
f'{12:06b}'

'001100'

12

'M'

12

'001100'

In [586]:
data = b'vidu is my hero'  # 15 bytes (r=0)
data = b'vidu is my boy'  # 14 bytes (r=2)
data = b'vidu is smart'  # 13 bytes (r=1)
type(data)

len(data)  # How many bytes in data
len(data) % 3  # r, Zero means no padding needed

bytes

13

1

## ⭐ Base64 Encoding Process

**Goal: Convert data (bytes) into string**

### Step 1: bytes -> binary string

In [587]:
binary_str = ''.join([f'{i:08b}' for i in data])

binary_str
len(binary_str)  # How many bits in data, is it a multiple of 6
len(binary_str) % 6  # Zero means we don't have a bad chunk (<6 bits) and no padding needed

'01110110011010010110010001110101001000000110100101110011001000000111001101101101011000010111001001110100'

104

2

### Step 2: Add padding bits if required

**Base64 perfectly deals groups of 3B:**  
3B = 24 bits -> 6, 6, 6, 6 [got 4 symbols]

**How many bytes remaining after splitting in groups of 3B? (r)**  
When total bytes is not multiple of 3, we have two scenarios:
- 1B = 8 bits -> 6, 2+4  [got 2 symbols, padding ==]
- 2B = 16 bits -> 6, 6, 4+2  [got 3 symbols, padding =] I.

**While encoding we have two padding scenarios:**
- bad chunk of 2 bits, +4 padding bits
- bad chunk of 4 bits, +2 padding bits

**While decoding we have two padding scenarios:**
- ==: 4 bits were added
- =: 2 bits were added

We must drop padding bits during decoding to obtain original data

In [588]:
count_equals = 0

r = len(data) % 3  # How many bytes are left if we make groups of 3B
if r == 1:
  count_equals = 2  # +4 bits, ==
if r == 2:
  count_equals = 1  # +2 bits, =

if count_equals > 0:
  binary_str += '0'*(2*count_equals)

count_equals  # Zero means no padding required
len(binary_str)  # Number of bits is now a multiple of 6
len(binary_str) % 6  # Must be zero now

2

108

0

### Step 3: Split binary string in chunks of 6 bits

In [589]:
# Problem: Split a str in chunks of length x

fruits = 'applemangolemon'
x = 5

cut_from = range(0, len(fruits), x)  # start, stop, step
list(cut_from)  # converting range to list (to print indices)

for i in cut_from:
  print(i, i+x, fruits[i:i+x])

[0, 5, 10]

0 5 apple
5 10 mango
10 15 lemon


In [590]:
chunks = [binary_str[i:i+6] for i in range(0, len(binary_str), 6)]
print(chunks)

['011101', '100110', '100101', '100100', '011101', '010010', '000001', '101001', '011100', '110010', '000001', '110011', '011011', '010110', '000101', '110010', '011101', '000000']


### Step 4: chunks -> nums (0-63)

In [591]:
nums = [int(chunk, 2) for chunk in chunks]
nums

[29, 38, 37, 36, 29, 18, 1, 41, 28, 50, 1, 51, 27, 22, 5, 50, 29, 0]

### Step 5: nums -> symbols

In [592]:
encoded_str = ''.join([base64_symbols[num] for num in nums])
encoded_str

'dmlkdSBpcyBzbWFydA'

### Step 6: Apply padding (=) to encoded string

In [593]:
encoded_str += '='*count_equals
encoded_str

'dmlkdSBpcyBzbWFydA=='

In [594]:
base64.b64encode(data).decode()

'dmlkdSBpcyBzbWFydA=='

In [595]:
# Our implementation matches built in implementation?
assert encoded_str == base64.b64encode(data).decode()  # Assert will raise error if False

## ⭐ Base64 Decoding Process

**Goal: Get back data (bytes) from encoded string**

### Step 1: Remove padding symbols (=)

In [596]:
encoded_str_og = encoded_str  # Save original encoded string before modifying
encoded_str_og

'dmlkdSBpcyBzbWFydA=='

In [597]:
count_equals = 0

if encoded_str[-2:] == "==":
  count_equals = 2
elif encoded_str[-1:] == "=":
  count_equals = 1

if count_equals > 0:
  encoded_str = encoded_str[:-count_equals]  # Drop last 1 or 2 characters (=)

count_equals  # Zero means no padding was added
encoded_str  # After padding removal

2

'dmlkdSBpcyBzbWFydA'

### Step 2: symbols -> nums

In [598]:
nums = [base64_symbols.index(symbol) for symbol in encoded_str]
nums

[29, 38, 37, 36, 29, 18, 1, 41, 28, 50, 1, 51, 27, 22, 5, 50, 29, 0]

### Step 3: nums -> chunks of 6 bits binary

In [599]:
chunks = [f'{num:06b}' for num in nums]
print(chunks)

['011101', '100110', '100101', '100100', '011101', '010010', '000001', '101001', '011100', '110010', '000001', '110011', '011011', '010110', '000101', '110010', '011101', '000000']


### Step 4: Join chunks to obtain full binary string

In [600]:
binary_str = ''.join(chunks)
binary_str

len(binary_str)  # How many bits in data, is it a multiple of 8
len(binary_str) % 8  # Non-zero means padding bits were added during encoding, which must be dropped now

'011101100110100101100100011101010010000001101001011100110010000001110011011011010110000101110010011101000000'

108

4

### Step 5: Drop padding bits

In [601]:
if count_equals > 0:
  binary_str = binary_str[:-2*count_equals]  # Drop last 2 or 4 bits

binary_str
len(binary_str)
len(binary_str) % 8  # Must be zero now

'01110110011010010110010001110101001000000110100101110011001000000111001101101101011000010111001001110100'

104

0

### Step 6: Split in chunks of 8 bits

In [602]:
chunks = [binary_str[i:i+8] for i in range(0, len(binary_str), 8)]
print(chunks)

['01110110', '01101001', '01100100', '01110101', '00100000', '01101001', '01110011', '00100000', '01110011', '01101101', '01100001', '01110010', '01110100']


### Step 7: chunks -> nums

In [603]:
nums = [int(chunk, 2) for chunk in chunks]
nums

[118, 105, 100, 117, 32, 105, 115, 32, 115, 109, 97, 114, 116]

### Step 8: nums -> bytes

In [604]:
decoded_data = bytes(nums)
decoded_data

b'vidu is smart'

In [605]:
base64.b64decode(encoded_str_og)

b'vidu is smart'

In [606]:
assert decoded_data == data  # Successfully decoded data?

# Our implementation matches built in implementation?
assert decoded_data == base64.b64decode(encoded_str_og)

# ✅ `base64_encode` and `base64_decode` functions

In [607]:
import string

base64_symbols = string.ascii_uppercase + string.ascii_lowercase + string.digits + '+/'

In [608]:
def base64_encode(data: bytes) -> str:
  # Step 1: bytes -> binary string
  binary_str = ''.join([f'{i:08b}' for i in data])

  # Step 2: Add padding bits if required
  count_equals = 0
  r = len(data) % 3
  if r == 1:
    count_equals = 2  # +4 bits, ==
  if r == 2:
    count_equals = 1  # +2 bits, =

  if count_equals > 0:
    binary_str += '0'*(2*count_equals)

  # Step 3: Split binary string in chunks of 6 bits
  chunks = [binary_str[i:i+6] for i in range(0, len(binary_str), 6)]

  # Step 4: chunks -> nums (0-63)
  nums = [int(chunk, 2) for chunk in chunks]

  # Step 5: nums -> symbols
  encoded_str = ''.join([base64_symbols[num] for num in nums])

  # Step 6: Apply padding (=) to encoded string
  encoded_str += '='*count_equals

  return encoded_str

In [609]:
def base64_decode(encoded_str: str) -> bytes:
  # Step 1: Remove padding symbols (=)
  count_equals = 0
  if encoded_str[-2:] == "==":
    count_equals = 2
  elif encoded_str[-1:] == "=":
    count_equals = 1

  if count_equals > 0:
    encoded_str = encoded_str[:-count_equals]

  # Step 2: symbols -> nums
  nums = [base64_symbols.index(symbol) for symbol in encoded_str]

  # Step 3: nums -> chunks of 6 bits binary
  chunks = [f'{num:06b}' for num in nums]

  # Step 4: Join chunks to obtain full binary string
  binary_str = ''.join(chunks)

  # Step 5: Drop padding bits
  if count_equals > 0:
    binary_str = binary_str[:-2*count_equals]

  # Step 6: Split in chunks of 8 bits
  chunks = [binary_str[i:i+8] for i in range(0, len(binary_str), 8)]

  # Step 7: chunks -> nums
  nums = [int(chunk, 2) for chunk in chunks]

  # Step 8: nums -> bytes
  decoded_data = bytes(nums)

  return decoded_data

### Testing

In [610]:
base64_encode(b'a')  # My encoding output
# Does it matches built in implementation?
assert base64_encode(b'a') == base64.b64encode(b'a').decode()

base64_encode(b'ab')
assert base64_encode(b'ab') == base64.b64encode(b'ab').decode()

base64_encode(b'abc')
assert base64_encode(b'abc') == base64.b64encode(b'abc').decode()

'YQ=='

'YWI='

'YWJj'

In [611]:
# My encoding is compatible with my decoding?
assert base64_decode(base64_encode(b'a')) == b'a'
assert base64_decode(base64_encode(b'ab')) == b'ab'
assert base64_decode(base64_encode(b'abc')) == b'abc'

In [612]:
# My encoding is compatible with built in decoding?
assert base64.b64decode(base64_encode(b'a')) == b'a'
assert base64.b64decode(base64_encode(b'ab')) == b'ab'
assert base64.b64decode(base64_encode(b'abc')) == b'abc'

In [613]:
import random
for i in range(10):
  random_bytes = random.randbytes(i)  # bytes of length i
  random_bytes
  # Can successfully encode and decode?
  assert base64_decode(base64_encode(random_bytes)) == random_bytes
  # Encoding output is same as built in implementation?
  assert base64_encode(random_bytes) == base64.b64encode(random_bytes).decode()

b''

b'='

b'\xc0\xdd'

b'\xe6\x980'

b'|kj\xf8'

b'O\x88\xb8\x07\x0e'

b'\x13\x80\xa2\xd1\x94?'

b'[\xcc\x88YL\xfd '

b'\xb6~\xef)"\xf7\xd1s'

b'\xd9\xd0\x07\xce\x1d\x99\xd2\xc0#'